In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import joblib
from tensorflow.keras.models import load_model
import json
from keras.layers import Dropout
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Default title text
# Load the dataset
dir = '/content/drive/MyDrive/capstone/'
df = pd.read_excel(dir+'DuplicateCleansing.xlsx')
# Preprocess text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Skillset'].fillna(''))

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['Skillset'].fillna(''))
sequence_lengths = [len(seq) for seq in sequences]

# Combine features
#maxlen = max(len(seq) for seq in sequences)
# X_text = pad_sequences(sequences, padding='post', maxlen=2000)

label_encoder_education = LabelEncoder()
df['Pendidikan'] = label_encoder_education.fit_transform(df['Pendidikan'])

scaler_experience = MinMaxScaler()
df['Pengalaman Kerja'] = scaler_experience.fit_transform(df[['Pengalaman Kerja']])

X = X_other = df[['Pengalaman Kerja', 'Pendidikan']]
# X = pd.concat([X_other, pd.DataFrame(X_text)], axis=1)

label_encoder_job_title = LabelEncoder()
y_encoded = label_encoder_job_title.fit_transform(df['Job Title'])

y_one_hot = to_categorical(y_encoded)

In [ ]:
X.shape, y_one_hot.shape

((99, 2), (99, 95))

In [ ]:
X.values

array([[0.3, 1. ],
       [0.2, 1. ],
       [0. , 1. ],
       [0.6, 1. ],
       [0.2, 1. ],
       [0.1, 4. ],
       [0.1, 4. ],
       [0. , 2. ],
       [0.2, 1. ],
       [0. , 1. ],
       [0.2, 0. ],
       [0.3, 2. ],
       [0. , 2. ],
       [0.5, 2. ],
       [0.3, 1. ],
       [0.2, 2. ],
       [0.4, 1. ],
       [0.3, 2. ],
       [0.1, 2. ],
       [0.2, 1. ],
       [0.2, 2. ],
       [0.2, 2. ],
       [0. , 2. ],
       [0. , 1. ],
       [0.1, 2. ],
       [0.1, 1. ],
       [0.2, 2. ],
       [0. , 2. ],
       [0.3, 2. ],
       [0.2, 2. ],
       [0.5, 2. ],
       [0.1, 1. ],
       [0. , 1. ],
       [0.1, 1. ],
       [0.3, 2. ],
       [0.1, 2. ],
       [0.1, 5. ],
       [0.2, 1. ],
       [0. , 2. ],
       [0.1, 1. ],
       [0.2, 2. ],
       [0.3, 1. ],
       [0.2, 2. ],
       [0.3, 2. ],
       [0.1, 2. ],
       [0. , 1. ],
       [0.2, 2. ],
       [0.5, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.5, 2. ],
       [0.1, 2. ],
       [0. ,

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Neural network model with Word Embedding
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder_job_title.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
3/3 [==============================] - 1s 120ms/step - loss: 4.5532 - accuracy: 0.0000e+00 - val_loss: 4.5555 - val_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 19ms/step - loss: 4.5526 - accuracy: 0.0127 - val_loss: 4.5615 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 18ms/step - loss: 4.5462 - accuracy: 0.0380 - val_loss: 4.5679 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 19ms/step - loss: 4.5439 - accuracy: 0.0253 - val_loss: 4.5751 - val_accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 0s 17ms/step - loss: 4.5384 - accuracy: 0.0506 - val_loss: 4.5831 - val_accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 0s 27ms/step - loss: 4.5371 - accuracy: 0.0253 - val_loss: 4.5925 - val_accuracy: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 0s 19ms/step - loss: 4.5325 - accuracy: 0.0506 - val_loss: 4.6028 - val_accuracy

In [ ]:
# Save the trained model and preprocessing components
model.save("/content/drive/MyDrive/capstone/nlp_model.h5")

# Save Tokenizer to JSON file
with open("/content/drive/MyDrive/capstone/nlp_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer.to_json())

# Save Label Encoder for Job Title
label_encoder_job_title_JobReq = "/content/drive/MyDrive/capstone/nlp_label_encoder_job_title.pkl"
joblib.dump(label_encoder_job_title, label_encoder_job_title_JobReq)

# Save Label Encoder for Education
label_encoder_education_JobReq = "/content/drive/MyDrive/capstone/nlp_label_encoder_education.pkl"
joblib.dump(label_encoder_education, label_encoder_education_JobReq)

# Save MinMaxScaler for Experience
scaler_experience_filename = "/content/drive/MyDrive/capstone/scaler_experience.pkl"
joblib.dump(scaler_experience, scaler_experience_filename)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/drive/MyDrive/capstone/scaler_experience.pkl']

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 2, 50)             78100     
                                                                 
 flatten_2 (Flatten)         (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               12928     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 95)                6175      
                                                                 
Total params: 105459 (411.95 KB)
Trainable params: 105

In [ ]:
df_user_input = pd.read_excel('/content/drive/MyDrive/capstone/User app dataset.xlsx')
model_loaded = load_model("/content/drive/MyDrive/capstone/nlp_model.h5")
tokenizer = Tokenizer()
with open("/content/drive/MyDrive/capstone/nlp_tokenizer.json", 'r') as f:
    tokenizer_config = json.load(f)
    tokenizer.word_index = tokenizer_config.get('word_index', {})

label_encoder_job_title = joblib.load("/content/drive/MyDrive/capstone/nlp_label_encoder_job_title.pkl")
label_encoder_education = joblib.load("/content/drive/MyDrive/capstone/nlp_label_encoder_education.pkl")
scaler_experience = joblib.load("/content/drive/MyDrive/capstone/scaler_experience.pkl")

df_user_input['Pendidikan'] = label_encoder_education.transform(df_user_input['Pendidikan'])
df_user_input['Pengalaman Kerja'] = scaler_experience.transform(df_user_input[['Pengalaman Kerja']]).reshape(-1, 1)

sequences_user_input = tokenizer.texts_to_sequences(df_user_input['Skillset'].fillna(''))
maxlen_train = max(len(seq) for seq in sequences_user_input)
X_text_user_input = pad_sequences(sequences_user_input, padding='post',maxlen=maxlen_train)

X_other_user_input = df_user_input[['Pengalaman Kerja', 'Pendidikan']]
X_user_input = pd.concat([X_other_user_input, pd.DataFrame(X_text_user_input)], axis=1)

In [ ]:
X_user_input.shape

(100, 2)

In [ ]:
X_user_input.values

array([[0.2, 2. ],
       [0.3, 2. ],
       [0.2, 2. ],
       [0.3, 2. ],
       [0.3, 1. ],
       [0.3, 2. ],
       [0.4, 2. ],
       [0.6, 1. ],
       [0.1, 2. ],
       [0.1, 5. ],
       [0.2, 1. ],
       [0.3, 2. ],
       [0.3, 2. ],
       [0.3, 2. ],
       [0.5, 2. ],
       [0.5, 2. ],
       [0.5, 2. ],
       [0.2, 2. ],
       [0.1, 2. ],
       [0.2, 2. ],
       [0.1, 1. ],
       [0.2, 2. ],
       [0.2, 2. ],
       [0.2, 1. ],
       [0.2, 2. ],
       [0.2, 5. ],
       [0.3, 5. ],
       [0.3, 4. ],
       [0.4, 1. ],
       [0.3, 2. ],
       [0.4, 1. ],
       [0.6, 2. ],
       [0.3, 2. ],
       [0.2, 2. ],
       [0.3, 2. ],
       [0. , 2. ],
       [0.2, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 2. ],
       [0.1, 1. ],
       [0.2, 1. ],
       [0.1, 2. ],
       [0.3, 5. ],
       [0.1,

In [ ]:
predictions = model_loaded.predict(X_user_input)

4/4 [==============================] - 0s 3ms/step


In [ ]:
# Decode predictions to job titles
predicted_labels = label_encoder_job_title.inverse_transform(np.argmax(predictions, axis=1))

# Add the predicted job titles to the user input DataFrame
df_user_input['predicted_job_title'] = predicted_labels

# Display the DataFrame with predicted job titles
print(df_user_input[['Skillset', 'predicted_job_title']])

                                   Skillset predicted_job_title
0   Microsoft Office , Node.js , Javascript          IT SUPPORT
1                         Javascript , HTML          IT SUPPORT
2                PHP, CSS, Microsoft Office          IT SUPPORT
3               HTML, CSS, Microsoft Office          IT SUPPORT
4                PHP, C+ , Microsoft Office          HRIS Staff
..                                      ...                 ...
95                      C, Javascript, HTML          IT SUPPORT
96                      C, Javascript, HTML          IT SUPPORT
97                      C, Javascript, HTML          IT SUPPORT
98                         Javascript, Java          IT SUPPORT
99               Python, Javascript, SQL, C          IT SUPPORT

[100 rows x 2 columns]


In [ ]:
df_user_input.to_csv('predict.csv', index=False)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
model_path = "model.tflite"
# Save the model to TFLite
with open(model_path, 'wb') as f:
    f.write(tflite_model)